Дан датасет с результатами проведённых баскетбольных матчей. Необходимо сделать прогноз, сколько всего очков будет набрано обеими командами в матче.

Для каждой игры в датасете есть несколько строк, описывающие хронологию матча. В колонке "info" одинаковые записи соответствуют одному и тому же матчу.

* TOTAL  - результат работы алгоритма предсказания владельцев базы данных.
* info   - данные о матче: где, кто с кем играл - текстовые данные.
* Минута - минута периода, в которую записываются текущие результаты игры.
* Секунда - секунда, в которую записываются текущие результаты игры.
* Общая минута - минута матча (не периода, а всего матча), в которую записываются текущие результаты игры.
* ftime  - время в секундах, прошедшее с начала матча (т.е. ftime = 'Общая минута'*60 + 'Секунда').
* Ком. 1 - сколько мячей забила первая команда на момент записи результатов
* Ком. 2 - сколько мячей забила вторая команда на момент записи результатов.

* fcount - общая сумма забитых мячей в игре. Причём в каждой из строк, принадлежащей конкретному матчу, указана именнно общая сумма забитых мячей.


Необходимо научить нейросеть предсказывать количество набранных очков в матче (fcount).

Примените Autokeras к датасету по ссылке. Разделите данные на текстовые и табличные, приведите к нужному формату, сделайте обучающую и проверочную выборки, записи таблицы не должны перемешиваться между собой в выборках. Запустите AutoModel из AutoKeras по двум типам данных, использовав TextInput() и StructuredDataInput(). Получите архитектуру лучшей модели.

Также код доступен и в 
https://github.com/dimarsoft/05.2023.ai.automl.git


In [ ]:
!pip install autokeras

In [ ]:
import pandas as pd
import gdown
from IPython.display import display
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pprint
import autokeras as ak
from pandas import DataFrame
import numpy as np



In [ ]:
def read_dataset() -> DataFrame:
    """
    Считать данные их файла
    :return:
        DataFrame: Данные
    """
  gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l10/basketball.csv', None, quiet=True)

  df = pd.read_csv('basketball.csv', encoding= 'cp1251', sep=';', header=0, index_col=0) # Загружаем базу
  return df


In [ ]:
def split_dataset(df: DataFrame) -> tuple[DataFrame, DataFrame, np.ndarray]:
    """
    Подготовка данных.
    Разделить на данные, текстовые данные, ответы

    :param df: Датафрейм со всеми данными
    :return:
        tuple[DataFrame данные, DataFrame текст, np.ndarray ответы]:

    """
    # ftime содержит "Минута", "Общая минута", "Секунда"
    # удаляем указанные столбцы
    data = df.drop(["info", "fcount", "Минута", "Общая минута", "Секунда"], axis=1)
    data = data.replace(',', '.', regex=True).astype(float)
    data = data.astype(float)

    df_text = df["info"].values
    y_train = np.array(df['fcount'].astype('int'))

    return data, df_text, y_train


In [ ]:
def scale_data(data: DataFrame):
    """
    Масштабируем данные, возможно это лучше, чем без него.
    :param data:
    :return:
    """
    scaler = MinMaxScaler() 
    # можно и StandardScaler()  пробовать

    # Нормализация значений столбцов 'TOTAL' и 'ftime' с помощью объекта scaler
    data[['TOTAL', 'ftime']] = scaler.fit_transform(data[['TOTAL', 'ftime']])


In [ ]:
def train(train_data: DataFrame, txt_data: DataFrame, y_train: np.ndarray) -> ak.AutoModel:
    """
    Получаем модель НС с помощью autokeras
    :param train_data:
    :param txt_data:
    :param y_train:
    :return:
    """
    # Разбиваем по выборкам
    xTrain, xTest, data_text_train, data_text_test, yTrain, yTest = train_test_split(
        train_data, txt_data, y_train, test_size=7450, shuffle=False)

    # Инициализация модели с несколькими входными и выходными данными.
    model = ak.AutoModel(
        inputs=[ak.TextInput(), ak.StructuredDataInput()],
        outputs=[
            ak.RegressionHead(loss="MAE", metrics=["mae"])
        ],
        overwrite=True,
        max_trials=16,
    )
    # Обучаем модель на подготовленных данных.
    model.fit(
        [data_text_train, xTrain],
        yTrain,
        epochs=10,
        validation_data=([data_text_test, xTest], yTest)
    )
    return model

In [ ]:
def print_model(model: ak.AutoModel):
    """
    Вывод информации о полученной модели НС
    :param model: Модель НС.
    :return:
    """
    best_model = model.export_model()
    # Получите архитектуру модели в виде JSON строки
    model_json = best_model.to_json()

    pprint.pprint(model_json)
    best_model.summary()

In [ ]:
def make_train():
    # читаем данные из файла
    df_from_file = read_dataset()

    # делим данные
    train_data, txt_data, y_train = split_dataset(df_from_file)

    # масштабируем данные
    scale_data(train_data)

    print(train_data, txt_data, y_train)

    # проверяем данные

    check_data(train_data)

    # получаем модель

    model = train(train_data, txt_data, y_train)

    print_model(model)

In [ ]:
# запуск задачи

make_train()



Trial 16 Complete [00h 03m 06s]
val_loss: 16.507484436035156

Best val_loss So Far: 14.316300392150879
Total elapsed time: 00h 52m 28s
Epoch 1/10
1407/1407 [==============================] - 21s 12ms/step - loss: 25.6907 - mae: 25.6907 - val_loss: 16.9259 - val_mae: 16.9259
Epoch 2/10
1407/1407 [==============================] - 13s 9ms/step - loss: 19.2844 - mae: 19.2844 - val_loss: 17.2177 - val_mae: 17.2177
Epoch 3/10
1407/1407 [==============================] - 14s 10ms/step - loss: 17.3726 - mae: 17.3726 - val_loss: 16.5940 - val_mae: 16.5940
Epoch 4/10
1407/1407 [==============================] - 14s 10ms/step - loss: 15.1073 - mae: 15.1073 - val_loss: 15.5598 - val_mae: 15.5598
Epoch 5/10
1407/1407 [==============================] - 13s 9ms/step - loss: 14.4557 - mae: 14.4557 - val_loss: 17.1641 - val_mae: 17.1641
Epoch 6/10
1407/1407 [==============================] - 13s 9ms/step - loss: 12.7638 - mae: 12.7638 - val_loss: 16.7467 - val_mae: 16.7467
Epoch 7/10
1407/1407 [======

('{"class_name": "Functional", "config": {"name": "model", "trainable": true, '
 '"layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": '
 '[null], "dtype": "string", "sparse": false, "ragged": false, "name": '
 '"input_1"}, "name": "input_1", "inbound_nodes": []}, {"class_name": '
 '"Custom>ExpandLastDim", "config": {"name": "expand_last_dim", "trainable": '
 'true, "dtype": "float32"}, "name": "expand_last_dim", "inbound_nodes": '
 '[[["input_1", 0, 0, {}]]]}, {"class_name": "TextVectorization", "config": '
 '{"name": "text_vectorization", "trainable": true, "dtype": "string", '
 '"max_tokens": 5000, "standardize": "lower_and_strip_punctuation", "split": '
 '"whitespace", "ngrams": null, "output_mode": "int", '
 '"output_sequence_length": 64, "pad_to_max_tokens": false, "sparse": false, '
 '"ragged": false, "vocabulary": null, "idf_weights": null, "encoding": '
 '"utf-8", "vocabulary_size": 1470}, "name": "text_vectorization", '
 '"inbound_nodes": [[["expand_last_di

In [ ]:
print_model(model)

('{"class_name": "Functional", "config": {"name": "model", "trainable": true, '
 '"layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": '
 '[null], "dtype": "string", "sparse": false, "ragged": false, "name": '
 '"input_1"}, "name": "input_1", "inbound_nodes": []}, {"class_name": '
 '"Custom>ExpandLastDim", "config": {"name": "expand_last_dim", "trainable": '
 'true, "dtype": "float32"}, "name": "expand_last_dim", "inbound_nodes": '
 '[[["input_1", 0, 0, {}]]]}, {"class_name": "TextVectorization", "config": '
 '{"name": "text_vectorization", "trainable": true, "dtype": "string", '
 '"max_tokens": 5000, "standardize": "lower_and_strip_punctuation", "split": '
 '"whitespace", "ngrams": null, "output_mode": "int", '
 '"output_sequence_length": 64, "pad_to_max_tokens": false, "sparse": false, '
 '"ragged": false, "vocabulary": null, "idf_weights": null, "encoding": '
 '"utf-8", "vocabulary_size": 1470}, "name": "text_vectorization", '
 '"inbound_nodes": [[["expand_last_di